# GLTF 格式教學 Scene & Node 篇

<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/scene-node.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[`Open in observablehq`](https://observablehq.com/@toonnyy8/gltf-scene-node/9)

![圖 1. scenes, nodes \[1\]](https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/gltfOverview-2.0.0b-scene-node.png)

圖 1. scenes, nodes \[1\]

## 簡介

### Scene 簡介

glTF 中可以儲存多個場景，而每個場景都會存在「scenes」屬性中，  
由「scene」屬性指向預設的場景，而「scenes」內會有「nodes」，  
表示每個場景所擁有的。

### Node 簡介

每個 node 都可以當成是場景中的一個物件，  
每個物件具有自己的名稱「name」，  
這些物件有可能是「mesh」、「skin」、「camera」、「light」等等，  
而物件之間還具有親子關係，並利用「children」用來記錄哪些 node 是自己的子代，  
最後還有「matrix」來記錄此物件的初始姿態。  
> ps.「matrix」可拆分成  
> 1. 代表偏移的「translation」
> 2. 代表旋轉的「rotation」
> 3. 代表縮放的「scale」

## 正式開始

### 載入檔案

In [0]:
%load https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/scene-node.ipynb

In [0]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/cube-channel.glb

--2020-04-06 13:26:25--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb [following]
--2020-04-06 13:26:25--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2280 (2.2K) [application/octet-stream]
Saving to: ‘cube.glb’

cube.glb            100%[===================>

In [0]:
glb_file = open('./cube-channel.glb', 'rb')

glb_file

<_io.BufferedReader name='./cube.glb'>

### 將檔案內容讀取出來

In [0]:
glb_bytes = glb_file.read()

In [0]:
glb_bytes

### 解析 header

0~3 Bytes 為「glTF」的 ASCII 編碼

In [0]:
glb_bytes[0:4].decode()

'glTF'

第 4~7 Bytes 應該會顯示 glTF 的版本

In [0]:
gltf_version = 0

for idx, curr in enumerate(glb_bytes[4:8]):
    gltf_version += curr * 256 ** idx

gltf_version

2

第 8~11 Bytes 則代表此檔案的 Byte 數

In [0]:
print(len(glb_bytes), "Bytes")

2280 Bytes


In [0]:
gltf_length = 0

for idx, curr in enumerate(glb_bytes[8:12]):
    gltf_length += curr * 256 ** idx

gltf_length

2280

### 解析 chunk

In [0]:
import json

def chunk_parser(chunk_type, chunk_data):
    return {
        "BIN\u0000": lambda:chunk_data,
        "JSON": lambda:json.loads(chunk_data.decode()),
    }[chunk_type]()

#### chunk 0 (JSON)

In [0]:
chunk = { 'type': "", 'length': 0, 'data': "" }

由 chunk 0 的 第 0~3 Bytes 知道 chunkLength 為多少

In [0]:
chunk["length"] = 0

for idx, curr in enumerate(glb_bytes[12:12 + 4]):
    chunk["length"] += curr * 256 ** idx

chunk["length"]

1412

由 chunk 0 的 第 4~7 Bytes 知道 chunkType 為 JSON

In [0]:
chunk["type"] = glb_bytes[12 + 4:12 + 8].decode()

chunk["type"]

'JSON'

知道了 chunkType 以及 chunkLength 就可以開始解析 chunkData 了

In [0]:
chunk["data"] = chunk_parser(
  chunk["type"],
  glb_bytes[12 + 8:12 + 8 + chunk["length"]]
)

In [0]:
chunk["data"]

#### 剩下的 chunk 解析的方式都相同，在此就直接使用迴圈處理

In [0]:
chunks = []
offset = 12
i = 0

while(offset < len(glb_bytes)):
    chunks.append({ "type": "", "length": 0, "data": "" })

    for idx, curr in enumerate(glb_bytes[offset:offset + 4]):
        chunks[i]["length"] += curr * 256 ** idx

    chunks[i]["type"] = glb_bytes[offset + 4:offset + 8].decode()

    chunks[i]["data"] = chunk_parser(
        chunks[i]["type"],
        glb_bytes[offset + 8:offset + 8 + chunks[i]["length"]]
    )
    offset += 8 + chunks[i]["length"]
    i += 1

In [0]:
chunks

最後將 chunks 整理一下

In [0]:
model = chunks[0]["data"]

for idx, buffer in enumerate(model['buffers']):
    buffer["data"] = chunks[idx + 1]["data"]

In [0]:
model

完成~~

## 參考

1. https://github.com/KhronosGroup/glTF